# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk import word_tokenize
import gzip
import pickle

In [4]:
import nltk
import re
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alire\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alire\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * from messages", engine)
X = df['message']
col_names=df.drop(['id','message','original','genre'], axis=1).columns
Y =df[col_names] 

In [7]:
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26210,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26211,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
                               ...                        
26210    The training demonstrated how to enhance micro...
26211    A suitable candidate has been selected and OCH...
26212    Proshika, operating in Cox's Bazar municipalit...
26213    Some 2,000 women protesting against the conduc...
26214    A radical shift in thinking came about as a re...
Name: message, Length: 26215, dtype: object

In [9]:
Y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26210,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    # Normalize text
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]"," ",text)
    
    # Tokenize text
    words = word_tokenize(text)
    
    # Remove stop words
    tokens = [w for w in words if w not in stopwords.words('english')]    
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip() # [WordNetLemmatizer().lemmatize(w) for w in tokens]
        clean_tokens.append(clean_tok)
    
    return clean_tokens

In [11]:
print(tokenize(X[3]))

['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
import pickle

In [13]:
# perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)

In [14]:
# Instantiate transformers and classifier
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer() 
clf = RandomForestClassifier()  # MultinomialNB()

# Fit and/or transform each to the data
vect_count = vect.fit_transform(X_train)
X_train_tf = tfidf.fit_transform(vect_count)
clf.fit(X_train_tf, y_train )  #tfidf.fit_transform(vect.fit_transform(X_test))

RandomForestClassifier()

In [15]:
# Transform test data
X_test_count = vect.transform(X_test)
X_test_tf = tfidf.transform(X_test_count)


# Predict test labels
y_pred = clf.predict(X_test_tf)

In [16]:
labels = np.unique(y_pred)
# confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
accuracy = (y_pred == y_test).mean()

print("Accuracy:", accuracy)

Accuracy: related                   0.827739
request                   0.903418
offer                     0.995117
aid_related               0.759994
medical_help              0.918981
medical_products          0.948276
search_and_rescue         0.975130
security                  0.980470
military                  0.970705
child_alone               1.000000
water                     0.953006
food                      0.934849
shelter                   0.926457
clothing                  0.986878
money                     0.979860
missing_people            0.988862
refugees                  0.966280
death                     0.956668
other_aid                 0.870156
infrastructure_related    0.937138
transport                 0.956057
buildings                 0.951022
electricity               0.978029
tools                     0.992981
hospitals                 0.990845
shops                     0.995575
aid_centers               0.988251
other_infrastructure      0.957431
weather_re

#### Creating model as pipeline and gread search

In [ ]:
pipeline = Pipeline([
         ('vect', CountVectorizer(tokenizer=tokenize)),
         ('tfidf', TfidfTransformer()),
         ('clf', RandomForestClassifier())])

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)


In [ ]:
accuracy = (y_pred == y_test).mean()

print("Accuracy:", accuracy)

#### Adding Feature union

In [ ]:
from custom_transformer import StartingVerbExtractor

In [ ]:
pipeline = Pipeline([
        ('features', FeatureUnion([
           ('text_pipeline', Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),('tfidf', TfidfTransformer())])),
            ('verb', StartingVerbExtractor())])),
         ('clf', RandomForestClassifier())
    ])


In [ ]:
accuracy = (y_pred == y_test).mean()

print("Accuracy:", accuracy)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
#print(classification_report(y_test, y_pred, zero_division=1, target_names=Y.columns))  #, target_names=target_names

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params().keys()

In [13]:
# hyerparameters grid to search within
parameters = [
    {'clf__bootstrap': [False, True],
     'clf__n_estimators': [80,90, 100, 110, 130],
     'clf__max_features': [0.6, 0.65, 0.7, 0.73, 0.751, 0.78, 0.8],
     'clf__min_samples_leaf': [10, 12, 14],
     'clf__min_samples_split': [3, 5, 7]
    },
]

model = GridSearchCV(pipeline, param_grid=parameters)

In [ ]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=1, target_names=Y.columns))  #, target_names=target_names

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [2]:
filename = 'classifier.gzip'
with gzip.open(filename, 'wb') as f:
    pickle.dump(model, f, protocol=0)
    
#pickle.dump(model,open('disaster_response_pickle_model.pkl','wb'))

NameError: name 'pickle' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.